In [198]:
import pandas as pd
import requests as reqs
import plotly.express as px
import plotly as plotly

In [187]:
df = pd.read_csv("18_19_Graduation_Rates.csv")
state_df = pd.read_csv("state_mapping.csv")

In [188]:
def reformat(rate):
    if(rate.startswith('LE') or rate.startswith('LT') or rate.startswith('GE')):
        return rate[2:]
    if('-' in rate):
        return rate[0:rate.index('-')]
    return rate

In [189]:
df = df[['STNAM','LEAID','SCHNAM','ALL_RATE_1819']] #only need state name, school district, school name, and graduation rate for now
df = df[df.ALL_RATE_1819 != 'PS'] #drop all rows for schools where data was protected

In [190]:
df['ALL_RATE_1819'] = df['ALL_RATE_1819'].apply(reformat) #remove letters and symbols from graduation rates
print(df)

         STNAM    LEAID                         SCHNAM ALL_RATE_1819
0      ALABAMA   100005        Albertville High School            94
1      ALABAMA   100006             Asbury High School            90
2      ALABAMA   100006            Douglas High School            90
3      ALABAMA   100006   Kate D Smith DAR High School            95
4      ALABAMA   100006  Brindlee Mountain High School            85
...        ...      ...                            ...           ...
22895  WYOMING  5605820                 Ten Sleep K-12            50
22896  WYOMING  5605830       Jackson Hole High School            95
22897  WYOMING  5605830      Summit Innovations School            80
22898  WYOMING  5606090              Upton High School            60
22899  WYOMING  5606240            Worland High School            85

[21425 rows x 4 columns]


In [ ]:
response = reqs.get("https://nces.ed.gov/opengis/rest/services/K12_School_Locations/EDGE_GEOCODE_PUBLICSCH_1819/MapServer/0/query?where=STATE='WY'&returnGeometry=false&returnIdsOnly=true&outFields=NAME,LOCALE,LAT,LON&f=json")
print(response.json())

In [141]:
#100901 - last objectID
i = 1
count_500 = 0
objectIds = ""
results_list = [];
while (i<100901):
    if(count_500 == 200):
        objectIds = objectIds[1:]
        request = "https://nces.ed.gov/opengis/rest/services/K12_School_Locations/EDGE_GEOCODE_PUBLICSCH_1819/MapServer/0/query?objectIds="
        request += objectIds
        request += "&returnGeometry=false&outFields=NAME,STATE,LAT,LON&f=json"
        response = reqs.get(request)
        for x in response.json()['features']:
            results_list.append(x['attributes'])
        objectIds = ""
        count_500 = 0
    else:
        objectIds = objectIds + "," +str(i)
        i+=1
        count_500+=1
        
objectIds = objectIds[1:]
request = "https://nces.ed.gov/opengis/rest/services/K12_School_Locations/EDGE_GEOCODE_PUBLICSCH_1920/MapServer/0/query?objectIds="
request += objectIds
request += "&returnGeometry=false&outFields=NAME,LOCALE,LAT,LON&f=json"
response = reqs.get(request)
for x in response.json()['features']:
    results_list.append(x['attributes'])

    


df2 = pd.DataFrame.from_dict(results_list)
print(df2)

                                       NAME STATE        LAT        LON LOCALE
0                 Albertville Middle School    AL  34.260194 -86.206174    NaN
1                   Albertville High School    AL  34.262226 -86.204884    NaN
2                   Evans Elementary School    AL  34.273318 -86.220144    NaN
3             Albertville Elementary School    AL  34.253251 -86.221834    NaN
4       Big Spring Lake Kindergarten School    AL  34.289849 -86.193289    NaN
...                                     ...   ...        ...        ...    ...
100895                      ROSA C. BENITEZ   NaN  18.213568 -66.044197     13
100896                     CHARLES E. MINER   NaN  18.208542 -66.055857     13
100897              JUSTINA VAZQUEZ MENDOZA   NaN  18.216539 -66.036440     13
100898         ANTONIO S PEDREIRA (PRE-TEC)   NaN  18.214828 -66.041534     13
100899                     HAYDEE CABALLERO   NaN  18.214892 -66.053336     13

[100900 rows x 5 columns]


In [142]:
df2.to_csv("school_geo_data.csv",index=False)

In [192]:
#Replace state names with abbreviations
df['STNAM'] = df['STNAM'].str.lower()
state_df['Name'] = state_df['Name'].str.lower()
df = pd.merge(df,state_df, left_on='STNAM', right_on='Name')


SyntaxError: invalid syntax (Temp/ipykernel_35504/2800208002.py, line 5)

In [193]:
df = df.drop(['STNAM','Name'], axis=1)
df.head()

,LEAID,SCHNAM,ALL_RATE_1819,Abbreviation
0,100005,Albertville High School,94,AL
1,100006,Asbury High School,90,AL
2,100006,Douglas High School,90,AL
3,100006,Kate D Smith DAR High School,95,AL
4,100006,Brindlee Mountain High School,85,AL


In [221]:
df3 = pd.merge(df,df2,left_on=["SCHNAM","Abbreviation"],right_on=["NAME","STATE"])
df3["Grad_Rate"] = df3["ALL_RATE_1819"].astype('int32')

In [222]:
fig = px.scatter_geo(df3,lat="LAT",lon="LON", color="Grad_Rate", hover_name="SCHNAM", color_continuous_scale="inferno",scope='usa', width = 1200, height=800)

In [223]:
import plotly.graph_objects as go # or plotly.express as px
#fig = go.Figure() # or any Plotly Express function e.g. px.bar(...)
# fig.add_trace( ... )
# fig.update_layout( ... )

import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
